# **1. Подключаем  google диск и загружаем на него файл computer_games.csv**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


импортируем pandas с алиасом pd

In [ ]:
import pandas as pd

# **1. Запускаем функцию с параметром пути к файлу .csv, обрабатываем исключения если файл не найден на диске или сбой загрузки**

In [ ]:
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"✅ Данные успешно загружены. Найдено {len(df)} строк.")
        return df
    except Exception as e:
        print(f"❌ Ошибка при загрузке файла: {e}")
        return None

df_plays = load_data('/content/drive/MyDrive/computer_games.csv')
df_plays

✅ Данные успешно загружены. Найдено 1095 строк.


,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
1,A-Train,Artdink,"Artdink, Maxis, Ocean Software",Vehicle Simulation Game,"Windows, Mac",1985
2,A-10 Cuba!,Parsoft Interactive,Activision,Flight simulator,"Windows, Mac","November 30, 1996"
3,A.D. 2044,R.M.P. Software,LK Avalon,Adventure,Microsoft Windows,"September 6, 1996"
4,A.D.A.M. Life's Greatest Mysteries,Columbia Healthcare Corporation,Columbia Healthcare Corporation,Educational,Microsoft Windows,1996
...,...,...,...,...,...,...
1090,688(I) Hunter/Killer,Sonalysts Inc.,Electronic Arts,Submarine simulator,Microsoft Windows,"July 4, 1997"
1091,7 Colors,Gamos Ltd.,Infogrames,Strategy,"Amiga, DOS",1991
1092,7554,Emobi Games,Emobi Games,First-person shooter,Microsoft Windows,"December 16, 2011"
1093,7th Legion,"Vision, Epic MegaGames",MicroProse,RTS,Microsoft Windows,"September 30, 1997"


# **Вариант 1: решим с помощью функции contains**

In [ ]:
"""
Расширим задачу и напишем функцию для поиска совпадений по любому столбцу
На вход функции подаем три аргумента:
df - имя DataFrame,
col - имя столбца в котором ищем совпадения,
substr - подстрока - шаблон
"""
def find_substr(df, col, substr):
  """
  для защиты от любопытного пользователя обработаем исключения
  (пока на начальном этапе общим классом Exception)
  """
  try:
    # если пользователь неверно набрал имя столбца
    if col not in list(df.columns):
      return 'Такого столбца не существует'

    # получаем новый DataFrame с применением фильтра
    filtered_df = df[df[col].str.contains(substr)]

    # если подстрока по шаблону не найдена: сообщаем об этом и выводим пустой DataFrame
    if len(filtered_df) == 0:
      return 'Совпадений не найдено'
    return filtered_df.drop('NameContainsSubstring', axis=1) #  вщзвращаем и удаляем последний столбец True/False
    # вывод исключения
  except Exception as e:
    raise e

find_substr(df_plays, 'Name', '1234567890')

'Совпадений не найдено'

In [ ]:
find_substr(df_plays, 'Name', 'A-')

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
1,A-Train,Artdink,"Artdink, Maxis, Ocean Software",Vehicle Simulation Game,"Windows, Mac",1985
2,A-10 Cuba!,Parsoft Interactive,Activision,Flight simulator,"Windows, Mac","November 30, 1996"


In [ ]:
find_substr(df_plays, 'Name_123', '1234567890')

'Такого столбца не существует'

# **Вариант 2: с использованием функции apply и lambda функции:**

In [ ]:
def find_substr(df, col, substr):
  try:
    if col not in list(df.columns):
      return 'Такого столбца не существует'

    df['NameContainsSubstring'] = df[col].apply(lambda x: substr in x)

    filtered_df = df[df['NameContainsSubstring']]
    if len(filtered_df) == 0:
      return 'Совпадений не найдено'
    return filtered_df.drop('NameContainsSubstring', axis=1)
  except Exception as e:
    raise e

find_substr(df_plays, 'Name', '1234567890')

'Совпадений не найдено'

In [ ]:
find_substr(df_plays, 'Names_123', 'A-')

'Такого столбца не существует'

In [ ]:
find_substr(df_plays, 'Developer', 'oo').head() # ограничиваем количество выгружаемых строк

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
62,Afterparty,Night School Studio,Night School Studio,Adventure,"Windows, macOS","September 29, 2017"
87,AGON,Private Moon Studios,Private Moon Studios,Adventure,Microsoft Windows,2003
98,Amnesia: A Machine for Pigs,The Chinese Room,Frictional Games,Horror game,"Microsoft Windows, Linux, macOS","September 10, 2013"
142,A Chair in a Room: Greenwater,Wolf & Wood Interactive Ltd,Wolf & Wood Interactive Ltd,Adventure,Microsoft Windows,"April 5, 2016"


# **Вариант 3:  с помощью генератора списков (list comprehension)**

In [ ]:
def find_substr(df, col, substr):
  try:
    if col not in list(df.columns):
      return 'Такого столбца не существует'

    df['NameContainsSubstring'] = [substr in x for x in df[col]]

    filtered_df = df[df['NameContainsSubstring']].drop('NameContainsSubstring', axis=1)
    if len(filtered_df) == 0:
      return 'Совпадений не найдено'
    return filtered_df
  except Exception as e:
    raise e

find_substr(df_plays, 'Genre', '123456789')

'Совпадений не найдено'

In [ ]:
find_substr(df_plays, 'Genre_123', '')

'Такого столбца не существует'

In [ ]:
find_substr(df_plays, 'Genre', 'Adventure').head()

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
3,A.D. 2044,R.M.P. Software,LK Avalon,Adventure,Microsoft Windows,"September 6, 1996"
7,A Girls Fabric Face,Stanislaw Truchowskit,TurnVex,Adventure,Microsoft Windows,"March 10, 2017"
21,Abzû,Giant Squid Studios,505 Games,"Adventure, art game, simulation",Microsoft Windows,"August 2, 2019"
30,Ace Ventura: The CD-Rom Game,7th Level,Bomico Entertainment Software,Graphic Adventure (Point-and-click),Microsoft Windows,"October 31, 1996"


# __Вариант 4  с использованием рекомендованной в задании функции find()__

In [ ]:
def find_substr(df, col, substr):
  try:
    if col not in list(df.columns):
      return 'Такого столбца не существует'
    filtered_df = df[df[col].str.find(substr) != -1].drop('NameContainsSubstring', axis=1)
    if len(filtered_df) == 0:
      return 'Совпадений не найдено'
    return filtered_df.head()
  except Exception as e:
    raise e

find_substr(df_plays, 'Genre', 'Adv')

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
3,A.D. 2044,R.M.P. Software,LK Avalon,Adventure,Microsoft Windows,"September 6, 1996"
7,A Girls Fabric Face,Stanislaw Truchowskit,TurnVex,Adventure,Microsoft Windows,"March 10, 2017"
21,Abzû,Giant Squid Studios,505 Games,"Adventure, art game, simulation",Microsoft Windows,"August 2, 2019"
30,Ace Ventura: The CD-Rom Game,7th Level,Bomico Entertainment Software,Graphic Adventure (Point-and-click),Microsoft Windows,"October 31, 1996"


In [ ]:
find_substr(df_plays, 'Genre', '12345')

'Совпадений не найдено'

In [ ]:
find_substr(df_plays, 'Genre123', 'Adv')

'Такого столбца не существует'

# **Вариант 5 с использованием match(). Но данный вариант пригоден для поиска совпадений ТОЛЬКО с начала строки**

In [ ]:
def find_substr(df, col, substr):
  try:
    if col not in list(df.columns):
      return 'Такого столбца не существует'

    filtered_df = df[df[col].str.match(substr)].drop('NameContainsSubstring', axis=1)

    if len(filtered_df) == 0:
      return 'Совпадений не найдено'
    return filtered_df.head()
  except Exception as e:
    raise e

find_substr(df_plays, 'Genre', 'Adventure')

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
3,A.D. 2044,R.M.P. Software,LK Avalon,Adventure,Microsoft Windows,"September 6, 1996"
7,A Girls Fabric Face,Stanislaw Truchowskit,TurnVex,Adventure,Microsoft Windows,"March 10, 2017"
21,Abzû,Giant Squid Studios,505 Games,"Adventure, art game, simulation",Microsoft Windows,"August 2, 2019"
49,Adam's Venture,Vertigo Digital Entertainment,Soedesco,Adventure,Microsoft Windows,2009


In [ ]:
find_substr(df_plays, 'Genre', 'dventure') #убираем первую букву шаблона, чтобы проверить совпадение не сначала строки

'Совпадений не найдено'

# **ВЫПОЛНИМ ЗАДАНИЕ 2: немного позволю его усложнить и написать код по выбору любого поля, не только Deleloper. Для этого также, как и предыдущих функциях добавим во входные параметры имя столбца. Отличие от первого задания: искать будем не по вхождению, а по полному соответствию**

In [ ]:
def find_games(df, col, str_value):
  try:
    if col not in list(df.columns):
      return 'Такого столбца не существует'

    # Добавим возможность регистронезависимого ввода
    filtered_df = df[df[col].str.lower() == str_value.lower().strip()]

    if len(filtered_df) == 0:
      return 'Совпадений не найдено'

    # Сбрасываем индекс для чистоты вывода
    return filtered_df.reset_index(drop=True).head()

  except Exception as e:
    raise e

find_games(df_plays, 'Developer', "bloober team")

,Name,Developer,Producer,Genre,Operating System,Date Released,NameContainsSubstring
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015",True
1,Layers of Fear,Bloober Team,Aspyr,Survival horror,"Microsoft Windows, Linux, macOS","February 16, 2016",False


In [ ]:
find_games(df_plays, 'Developer', "bloober")

'Совпадений не найдено'

In [ ]:
find_games(df_plays, 'Developer123', "bloober team")

'Такого столбца не существует'

In [ ]:
find_games(df_plays, 'Operating System', "microsoft windows")

,Name,Developer,Producer,Genre,Operating System,Date Released,NameContainsSubstring
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015",True
1,A.D. 2044,R.M.P. Software,LK Avalon,Adventure,Microsoft Windows,"September 6, 1996",True
2,A.D.A.M. Life's Greatest Mysteries,Columbia Healthcare Corporation,Columbia Healthcare Corporation,Educational,Microsoft Windows,1996,False
3,A.D.A.M. The Inside Story,Columbia Healthcare Corporation,Columbia Healthcare Corporation,Educational,Microsoft Windows,1995,False
4,A.R.E.S.: Extinction Agenda,Extend Studio,"Origo Games, Aksys Games","Action, platform",Microsoft Windows,"December 14, 2010",False


# **Очень важное замечание: абсолютно все наши функции похожи как две капли воды, за исключением способа фильтрации. Если нам реально придется писать приложение для production, где будут использоваться разные методы фильтрации, то есть резон написать пользовательсткий декоратор и обернуть методы фильтрации в качестве функций в него**

In [ ]:
def search_strategy(strategy_func):
    def decorator(func):
        def wrapper(df, col, substr):
            try:
                if col not in list(df.columns):
                    return 'Такого столбца не существует'
                # Используем стратегию поиска
                filtered_df = strategy_func(df, col, substr)
                if len(filtered_df) == 0:
                    return 'Совпадений не найдено'
                return filtered_df
            except Exception as e:
                raise e
        return wrapper
    return decorator

# Примеры стратегий поиска
def match_strategy(df, col, substr):
    return df[df[col].str.match(substr)].drop('NameContainsSubstring', axis=1)

def find_strategy(df, col, substr):
    return df[df[col].str.find(substr) != -1].drop('NameContainsSubstring', axis=1)

def contains_strategy(df, col, substr):
    df['NameContainsSubstring'] = [substr in x for x in df[col]]
    return df[df['NameContainsSubstring']].drop('NameContainsSubstring', axis=1)

Проверим работу декоратора

1. match()

In [ ]:
@search_strategy(match_strategy)
def find_substr_match(df, col, substr):
    pass

find_substr_match(df_plays, 'Name', 'A-')

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
1,A-Train,Artdink,"Artdink, Maxis, Ocean Software",Vehicle Simulation Game,"Windows, Mac",1985
2,A-10 Cuba!,Parsoft Interactive,Activision,Flight simulator,"Windows, Mac","November 30, 1996"


In [ ]:
find_substr_match(df_plays, 'Name', '123567')

'Совпадений не найдено'

In [ ]:
find_substr_match(df_plays, 'Name123', 'A-')

'Такого столбца не существует'

2. find()

In [ ]:
@search_strategy(find_strategy)
def find_substr_find(df, col, substr):
    pass

find_substr_find(df_plays, 'Operating System', 'Microsoft')

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
3,A.D. 2044,R.M.P. Software,LK Avalon,Adventure,Microsoft Windows,"September 6, 1996"
4,A.D.A.M. Life's Greatest Mysteries,Columbia Healthcare Corporation,Columbia Healthcare Corporation,Educational,Microsoft Windows,1996
5,A.D.A.M. The Inside Story,Columbia Healthcare Corporation,Columbia Healthcare Corporation,Educational,Microsoft Windows,1995
6,A.R.E.S.: Extinction Agenda,Extend Studio,"Origo Games, Aksys Games","Action, platform",Microsoft Windows,"December 14, 2010"
...,...,...,...,...,...,...
1087,5,Ram,Visual Art's,"Eroge, visual novel",Microsoft Windows,"July 25, 2008"
1090,688(I) Hunter/Killer,Sonalysts Inc.,Electronic Arts,Submarine simulator,Microsoft Windows,"July 4, 1997"
1092,7554,Emobi Games,Emobi Games,First-person shooter,Microsoft Windows,"December 16, 2011"
1093,7th Legion,"Vision, Epic MegaGames",MicroProse,RTS,Microsoft Windows,"September 30, 1997"


3. contains()

In [ ]:
@search_strategy(contains_strategy)
def find_substr_contains(df, col, substr):
    pass

find_substr_contains(df_plays, 'Developer', 'oo')

,Name,Developer,Producer,Genre,Operating System,Date Released
0,A-Men 2,Bloober Team,Bloober Team,"Adventure, Puzzle",Microsoft Windows,"June 24, 2015"
62,Afterparty,Night School Studio,Night School Studio,Adventure,"Windows, macOS","September 29, 2017"
87,AGON,Private Moon Studios,Private Moon Studios,Adventure,Microsoft Windows,2003
98,Amnesia: A Machine for Pigs,The Chinese Room,Frictional Games,Horror game,"Microsoft Windows, Linux, macOS","September 10, 2013"
142,A Chair in a Room: Greenwater,Wolf & Wood Interactive Ltd,Wolf & Wood Interactive Ltd,Adventure,Microsoft Windows,"April 5, 2016"
263,Command & Conquer,Westwood Studios,Virgin Interactive,RTS,"MS-DOS, Microsoft Windows","August 31, 1995"
264,Command & Conquer: Red Alert,Westwood Studios,Virgin Interactive,RTS,"MS-DOS, Microsoft Windows","November 22, 1996"
266,Command & Conquer: Red Alert 2,Westwood Pacific,EA Games,RTS,Microsoft Windows,"October 26, 2000"
268,Command & Conquer: Renegade,Westwood Studios,EA Games,Tactical shooter,Microsoft Windows,"February 26, 2002"
269,Command & Conquer: Tiberian Sun,Westwood Studios,Electronic Arts,RTS,Microsoft Windows,"August 27, 1999"
